In [104]:
cd D:\7.pythoncode\Quantitative-Trading\1119

D:\7.pythoncode\Quantitative-Trading\1119


# 單筆股票作活儲 

+ 利率 0.1%
+ 大於小於 300 日買賣
+ 如果沒持有股 & 無法進行買賣 = 丟入存款

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [106]:
# import other ipynb
import import_ipynb
from figure import show_bollinger, bollinger_bands, funding_changes, show_returns, draw_bubble_WL, draw_2D_WL, draw_PL_BEP_Figure
from transaction import *
from profit_loss import profit_loss_analysis
from helper_func import *

In [107]:
get_one_stock('2882.TW')

,open,high,low,close,adjclose,volume,ticker,middle,std,lower,upper,B,returns
2017-10-02,47.980701,48.279301,47.980701,48.080200,42.198639,6232015,2882.TW,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-03,48.030499,48.378899,47.831402,48.130001,42.242348,12655478,2882.TW,NaN,NaN,NaN,NaN,NaN,0.001036
2017-10-05,48.080200,48.378899,47.980701,48.329102,42.417095,6081554,2882.TW,NaN,NaN,NaN,NaN,NaN,0.004137
2017-10-06,48.329102,48.577900,48.279301,48.478401,42.548130,7770016,2882.TW,NaN,NaN,NaN,NaN,NaN,0.003089
2017-10-11,48.926399,49.075699,48.876598,48.976101,42.984947,20468960,2882.TW,NaN,NaN,NaN,NaN,NaN,0.010266
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-24,38.799999,38.799999,38.000000,38.000000,38.000000,26794853,2882.TW,39.5575,0.507905,38.541691,40.573309,-0.266630,-0.025641
2020-09-25,38.049999,38.349998,38.000000,38.099998,38.099998,12900119,2882.TW,39.4650,0.593850,38.277299,40.652700,-0.074640,0.002632
2020-09-28,38.200001,38.549999,38.200001,38.549999,38.549999,8827394,2882.TW,39.4000,0.620060,38.159881,40.640119,0.157291,0.011811
2020-09-29,38.650002,38.799999,38.599998,38.700001,38.700001,7720480,2882.TW,39.3425,0.629385,38.083730,40.601269,0.244791,0.003891


In [108]:
stock_data = get_one_stock('2882.TW' ,days=300,start_date='10/01/2010', end_date='10/01/2020')
account = init_account()
account['capital'] = 1000000
account['init_capital'] = account['capital']
account['deposit'] = 0 # 多少錢放在定存
account['savings_month'] = 0 # 存了多少個月
rate = init_rate()
print(account  ,   rate)


{'shares': 0.0, 'cost': 0.0, 'buy_times': 0.0, 'earn': 0.0, 'sell_times': 0.0, 'capital': 1000000, 'per_trans': 1000, 'init_capital': 1000000, 'deposit': 0, 'savings_month': 0} {'handling_fee': 0.001425, 'trans_tax': 0.003, 'slippage': 1}


In [109]:
# 把存款拿出來
def draw_deposit(account):
    if account['savings_month'] > 0: # 假設有存銀行
        account['capital'] =  account['capital'] + account['deposit'] + account['deposit']*(0.001)*account['savings_month']/12 
        print( 'get ' , account['deposit'] + account['deposit']*(0.001)*account['savings_month']/12 )
        account['deposit'] = 0
        account['savings_month'] = 0
    return account

# 更新存款
def update_deposit(account):
    # 首存
    if account['deposit'] == 0:
        account['deposit'] =  account['capital']
        account['captial'] = 0
        account['savings_month'] = 0   
    else:
        account['savings_month'] +=  1   
    return account

# 看活存現金 + 手頭現金能不能買
def can_buy(account , price , rate):
    actuall_price = account['per_trans'] * (price + rate['slippage'])
    actuall_price += actuall_price * rate['handling_fee']
    if account['capital'] + account['deposit'] +  \
        account['deposit']*(0.001)*account['savings_month']/12 > actuall_price:
        return True
    else:
        return False

# all in 
def all_buy(account, price, rate):
    
    actuall_price = account['per_trans'] * (price + rate['slippage'])
    actuall_price += actuall_price * rate['handling_fee']
    buy_count = 0
    while account['capital'] > actuall_price:
        buy_count += 1
        account['cost'] += actuall_price 
        account['shares'] += account['per_trans']
        account['capital'] -= actuall_price

    account['buy_times'] += 1

    return account, actuall_price*buy_count


In [110]:
len(stock_data)

2459

In [111]:
records = []
for day in range( 300,len(stock_data)):
    date = stock_data.index.date[day]
    if check_if_last_day_of_month(date):
        # 判斷信號
        if   stock_data.close[day] > stock_data.middle[day]:
            sign = 'BUY'
        elif  stock_data.close[day] < stock_data.middle[day]:
            sign = 'SELL'

        # 做動作
       
        _capital = account['capital']
        if sign == 'BUY' and account['shares'] == 0 and can_buy( account , stock_data.close[day] , rate ):
            #print(sign)
            #print(f'pre {_capital}')
            account = draw_deposit(account)
            _capital = account['capital']
            #print(f'after {_capital}')
            account , actuall_price = all_buy( account  ,stock_data.close[day] ,rate )
            records.append( [ date , sign , _capital , account['capital'] , account['shares'] ,stock_data.close[day] , actuall_price ])
            
        elif sign == 'SELL' and account['shares'] > 0:
            #print(sign)
            account , actuall_price = sell( account , stock_data.close[day] ,rate )
            records.append( [ date , sign , _capital , account['capital'] , account['shares'] ,stock_data.close[day] , actuall_price ])
            
        else :
            if account['deposit'] == 0 and account['shares']==0 :
                
                #print(f'first save')
                account['deposit'] =  account['capital']
                account['capital'] = 0
               
                account['savings_month'] = 0   
            else :
                #print('gain')
                account['savings_month'] +=  1   
    
            #account = update_deposit(account)
        #print(account)
# 最後賣了
if account['shares'] != 0: 
    account,actuall_price = sell( account , stock_data.close[day] ,rate )
    records.append( [ date , sign , _capital , account['capital'] , account['shares'] ,stock_data.close[day] , actuall_price ])   
        

first save
gain
gain
gain
gain
gain
gain
get  1000500.0
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
first save
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
gain
get  1671434.507128343
first save
gain
gain
gain
gain
gain
gain
gain
gain
get  1552195.1958651056
first save
gain


In [112]:
account

{'shares': 0,
 'cost': 4194836.264030119,
 'buy_times': 3.0,
 'earn': 4653056.727133864,
 'sell_times': 3.0,
 'capital': 0,
 'per_trans': 1000,
 'init_capital': 1000000,
 'deposit': 1462535.659526221,
 'savings_month': 1}

In [113]:
trans = pd.DataFrame( records , columns=['日期', '動作', '交易前總資金' ,'交易後總資金', '持有股', '交易每股金額', '實際買賣金額'] ).round(2)

In [114]:
trans.head(50)

,日期,動作,交易前總資金,交易後總資金,持有股,交易每股金額,實際買賣金額
0,2012-11-30,BUY,1000500.00,7000.75,35000.0,27.35,993499.25
1,2015-08-31,SELL,7000.75,1668653.42,0.0,46.69,1661652.66
2,2018-08-31,BUY,1671434.51,14895.64,31000.0,52.36,1656538.87
3,2018-10-31,SELL,14895.64,1551161.09,0.0,48.78,1536265.45
4,2020-06-30,BUY,1552195.20,7397.05,36000.0,41.85,1544798.15
5,2020-07-31,SELL,7397.05,1462535.66,0.0,39.60,1455138.61
